# Лекция 12: диспетчеризация урлов и представления в Django

## Немного принципов
* KISS
* DRY
* NIH
* YAGNI

## Диспетчеризация урлов

### ЧПУ

ЧПУ - (человеко-понятные урлы) - идея формирования "хороших" урлов, которые были бы удобнее, в первую очередь, для человека, а не машины:  
  * содержат читаемые слова и даты  
  * латинский алфавит  
  * показывают вложенность директорий сайта  
  * отражают название сущности, на которую они ссылаются  
  * избегаются технические части текста (расширения, спецсимволы)

### URLconf

* URL dispatcher - (диспетчеризация урлов, routes, роуты) - способ задания схемы отображения из урла запроса на обработчик запроса.
* URLconf - неформальное название задаваемого разработчиком отображения с помощью встроенного в Django механизма.
* Содержится в файлах urls.py.
* Отображение задаётся с помощью кода на Python, которому указывается простое регулярное выражение (описание урла) и обрабатывающий объект-представление.
* URLConf могут вкладываться один в другой - обычно в корневой URLConfg проекта включаются URLConf дочерних приложений.

Пример:

In [ ]:
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^articles/2003/$', views.special_case_2003),
    url(r'^articles/([0-9]{4})/$', views.year_archive, name='news-year-archive'),
    url(r'^articles/([0-9]{4})/([0-9]{2})/$', views.month_archive),
    url(r'^articles/([0-9]{4})/([0-9]{2})/([0-9]+)/$', views.article_detail),
]

Иногда нужно искать url по шаблону (чтобы было проще вносить изменения и меньше повторяться).

Для этого существуют механизмы как в Python коде:

In [ ]:
reverse('news-year-archive', args=(year,))

Так и в шаблонах:

<a href="{% url 'news-year-archive' 2012 %}">2012 Archive</a>

* Подробнее про возможности URLConf:
  * https://docs.djangoproject.com/en/1.8/topics/http/urls/
  * http://djbook.ru/rel1.8/topics/http/urls.html

## Представления

* Представление - (view, вид) - функция на языке Python, в Django с помощью их реализуются обработчики запросов.  
* Принимает на вход объект-запрос (*django.http.HttpRequest*) и возвращает объект-ответ (*django.http.HttpResponse*).
* Ответ должен задаваться во view, обычно он генерирует html.

Пример:

In [ ]:
from django.http import HttpResponse
import datetime

def current_datetime(request):
    now = datetime.datetime.now()
    html = "<html><body>It is now %s.</body></html>" % now
    return HttpResponse(html)

In [ ]:
from django.http import HttpResponse, HttpResponseNotFound

def my_view(request):
    # ...
    if foo:
        return HttpResponseNotFound('<h1>Page not found</h1>')
    else:
        return HttpResponse('<h1>Page was found</h1>')

In [ ]:
# default will be used or specify your own 404.html template
from django.http import Http404
from django.shortcuts import render_to_response
from polls.models import Poll

def detail(request, poll_id):
    try:
        p = Poll.objects.get(pk=poll_id)
    except Poll.DoesNotExist:
        raise Http404("Poll does not exist")
    return render_to_response('polls/detail.html', {'poll': p})

### Вспомогательные функции

* Лежат в django.shortcuts.
* Позволяют упростить ряд рутинных или часто нужных действий.

#### render 
Рендерит шаблон и создаёт сооветствующий HttpResponse.

In [ ]:
from django.shortcuts import render

def my_view(request):
    # View code here...
    return render(request, 'myapp/index.html', {"foo": "bar"},
        content_type="application/xhtml+xml")

In [ ]:
from django.http import HttpResponse
from django.template import RequestContext, loader

def my_view(request):
    # View code here...
    t = loader.get_template('myapp/index.html')
    c = RequestContext(request, {'foo': 'bar'})
    return HttpResponse(t.render(c),
        content_type="application/xhtml+xml")

#### redirect

Позволяет выполнить редирект путём возвращения специального ответа, создает HttpResponseRedirect на соответствующий url.

In [ ]:
from django.shortcuts import redirect

def my_view(request):
    ...
    object = MyModel.objects.get(...)
    return redirect(object)

In [ ]:
def my_view(request):
    ...
    return redirect('some-view-name', foo='bar')

In [ ]:
def my_view(request):
    ...
    return redirect('/some/url/')

In [ ]:
def my_view(request):
    ...
    return redirect('http://example.com/')

#### get_object_or_404

Оборачивает получение объекта из базы (get): если не найдет, то вернёт ответ 404.

In [ ]:
from django.shortcuts import get_object_or_404

def my_view(request):
    my_object = get_object_or_404(MyModel, pk=1)

In [ ]:
from django.http import Http404

def my_view(request):
    try:
        my_object = MyModel.objects.get(pk=1)
    except MyModel.DoesNotExist:
        raise Http404("No MyModel matches the given query.")

In [ ]:
queryset = Book.objects.filter(title__startswith='M')
get_object_or_404(queryset, pk=1)

#### get_list_or_404

Как предыдущий, только для нескольких объектов (filter).

In [ ]:
from django.shortcuts import get_list_or_404

def my_view(request):
    my_objects = get_list_or_404(MyModel, published=True)

In [ ]:
from django.http import Http404

def my_view(request):
    my_objects = list(MyModel.objects.filter(published=True))
    if not my_objects:
        raise Http404("No MyModel matches the given query.")

Подробнее про представления:
* https://docs.djangoproject.com/en/1.8/#the-view-layer
* http://djbook.ru/rel1.8/#the-view-layer

На оставшейся части лекции рассматриваем основные части архитектуры большого веб-приложения.